In [11]:
from CandidateGenerator import CandidateGenerator as CG
"Import necessary to fix atoms in the slab"
from ase.constraints import FixAtoms
from ase import Atoms
from ase.io import read
from generate_ceo2 import create_ceo2_111 as ceo2Slab
import numpy as np
from ase.visualize import view
from ase.calculators.emt import EMT
from ase.db import connect
from pathlib import Path

"--------------------------Define the fitness fucntion for your atoms-----------------------"

def fitness_function(atoms,env)-> float:
    o2_energy = read('test_data/o2_pw91.traj').get_potential_energy()
    reference=0.0# read('test_data/pt6o0_fs_meta.traj@:')[0].get_potential_energy()

    at_num= atoms.get_atomic_numbers()
    o_num = np.count_nonzero(at_num == 8)
    ce_num= np.count_nonzero(at_num == 58)

    o_num = o_num - (2* ce_num)
    fre = atoms.get_potential_energy() - reference - o_num * (0.5 * o2_energy) - o_num*env*0.5
    return fre

"--------------------------------Database Initialization -----------------------------------"

if Path('databaseGA.db').is_file():
    Path('databaseGA.db').unlink()
db = connect('databaseGA.db')


"---------------------------Generate static part of the system------------------------------"
slab = ceo2Slab(number_of_layers=1,repetitions = (5,5,1),vacuum=12.0)
a = 12.0
slab = Atoms('Pt',cell=[a,a,a],
             pbc=True)
c = FixAtoms(indices=np.arange(len(slab)))
slab.set_constraint(c)
"If slab atoms arent fixed they will be generated  for all structures, but will be relaxed"


"---------------------------Define Considered Stoichiometries--------------------------------"

atoms_array = ['Pt6O1','Pt6O2','Pt6O3','Pt6O6']
#atoms_array = [Atoms('PtO1'),Atoms('Pt1O2'),Atoms('Pt1O3')]
#atoms_array = [[78]*2+[8]*1,[78,8,8],[78,8,8,8]]

"------------------------------Generate Starting Population----------------------------------"

candidateGenerator = CG(slab, atoms_array)


population = 5
for i in range(len(atoms_array)):
    for j in range(population):
        db.write(candidateGenerator.get_candidate_by_position(i), stoichiometry = atoms_array[i],relaxed=False)

"-----------------------------------Define Calculator----------------------------------------"

calc = EMT()

"---------------------------------Relax initial structures-----------------------------------"
for row in db.select(relaxed = False):
    atoms = row.toatoms()
    atoms.calc = calc
    atoms.get_potential_energy()
    db.write(atoms=atoms,stoichiometry = row.stoichiometry, relaxed = True)

"--------------------------------------Find better stoich to eval-----------------------------"

envs = [100,50,25,15,0.5,0.25,0.0,-0.5,-1.0,-2.0,-3.0,-1000.0]

fitness = 1000.0
stc='aaa'
for env in envs:
    fitness = 1000.0
    for row in db.select(relaxed = True):
        atoms = row.toatoms()
        new_fitness = fitness_function(atoms,env)
        if(new_fitness<fitness):
            fitness = new_fitness
            stc = row.stoichiometry
    print("{} -- {}".format(env,stc) )
    


100 -- Pt6O6
50 -- Pt6O6
25 -- Pt6O6
15 -- Pt6O6
0.5 -- Pt6O1
0.25 -- Pt6O1
0.0 -- Pt6O1
-0.5 -- Pt6O1
-1.0 -- Pt6O1
-2.0 -- Pt6O1
-3.0 -- Pt6O1
-1000.0 -- Pt6O1
